In [ ]:
# Импортируем необходимые модули проекта и библиотеки
from src.data_utils import dataset_cleaner
from src import next_token_dataset
from src.next_token_dataset import get_datasets, collate_fn
from src.lstm_model import LSTMTokenPredictor
from src.lstm_model_att import LSTMAttentionTokenPredictor, LSTMCausalAttentionPredictor
#from src.lstm_train import training_loop
from src.lstm_train_up import training_loop

import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Читаем и чистим исходный текст, сохраняем очищенный датасет
dataset_cleaner("data/tweets.txt", "data/tweets_cleaned.csv", 500000)

In [ ]:
# Загружаем токенизатор (подберите подходящий для вашего языка и задачи)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Получаем токенезированные датасеты
train_dataset, val_dataset = get_datasets(tokenizer, 'data/tweets_cleaned_small.csv')

# Создаём DataLoader'ы
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, collate_fn=collate_fn, num_workers=4, pin_memory=True)

In [ ]:
# Размер словаря
vocab_size = tokenizer.vocab_size 

# Создаем объект модели
#model = LSTMTokenPredictor(vocab_size=vocab_size)
#model = LSTMAttentionTokenPredictor(vocab_size=vocab_size)
model = LSTMCausalAttentionPredictor(vocab_size=vocab_size)

# Запускаем обучение модели 
train_losses, val_losses, rouge_scores = training_loop(model, tokenizer, train_loader, val_loader, n_epochs=10)

In [ ]:
# Отрисовка графиков

epochs = np.arange(1, len(train_losses) + 1)

plt.figure(figsize=(10, 6))

# --- Левая ось: train / val loss ---
plt.plot(epochs, train_losses, label='Train Loss', color='tab:blue', marker='o')
plt.plot(epochs, val_losses, label='Validation Loss', color='tab:orange', marker='o')

plt.xlabel("Epoch")
plt.ylabel("Loss", color='tab:blue')
plt.tick_params(axis='y', labelcolor='tab:blue')

# --- Правая ось: ROUGE ---
ax2 = plt.gca().twinx()
ax2.plot(epochs, rouge_scores, label='ROUGE-1', color='tab:green', marker='s')
ax2.set_ylabel("ROUGE-1", color='tab:green')
ax2.tick_params(axis='y', labelcolor='tab:green')

# --- Общее оформление ---
plt.title("Training Progress: Loss and ROUGE-1 per Epoch")
plt.grid(True, alpha=0.3)

# Совмещённая легенда для обеих осей
lines, labels = plt.gca().get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines + lines2, labels + labels2, loc='upper right')

plt.tight_layout()
plt.show()

In [5]:
# Подготовка датасета/даталоадера для трансформера
from src.dist_gpt import prepare_gpt_tokenizer, load_gpt_model, evaluate_transformer_rouge
from src.next_token_dataset import PredictDataset, collate_fn, choose_max_len
import pandas as pd

# Загружаем токенизатор GPT-2
tokenizer = prepare_gpt_tokenizer("distilgpt2")

# Загружаем тестовый датасет из ранее сохранённого CSV
test_df = pd.read_csv("data/test.csv")

# Получаем ту же max_len, что при обучении LSTM (можно захардкодить, если известно)
max_len = choose_max_len(test_df["clean_text"].dropna().astype(str).tolist())

# Создаём датасет
test_dataset = PredictDataset(test_df["clean_text"].tolist(), tokenizer, max_len)

# DataLoader
test_loader = DataLoader(
    test_dataset,
    batch_size=16,  # чуть меньше, чтобы не перегружать память GPT-2
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True
)


In [ ]:
# Transformer

model = load_gpt_model("distilgpt2")
results, preds, refs = evaluate_transformer_rouge(model, tokenizer, test_loader,
                                                 sample_limit=None,
                                                 generation_kwargs=dict(temperature=0.8, top_k=50, top_p=0.95, do_sample=True),
                                                 verbose_examples=3)

Eval transformer ROUGE:   0%|          | 0/3110 [00:00<?, ?it/s]c:\Users\smoly\OneDrive\Рабочий стол\Курс DL\.fpenv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
